### Welcome to Week 6 Day 3!

Let's experiment with a bunch more MCP Servers

In [ ]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace
from agents.mcp import MCPServerStdio, MCPServerSse
import os
from IPython.display import Markdown, display
from datetime import datetime
load_dotenv(override=True)

### The first type of MCP Server: runs locally, everything local

Here's a really interesting one: a knowledge-graph based memory.

It's a persistent memory store of entities, observations about them, and relationships between them.

https://github.com/modelcontextprotocol/servers/tree/main/src/memory


In [ ]:
params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-memory"]}

async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()

mcp_tools

In [3]:
instructions = "You use your entity tools as a persistent memory to store and recall information about your conversations."
request = f"My name's Ed. I'm an LLM engineer. I'm teaching a course about AI Agents, including the incredible MCP protocol. \
MCP is a protocol for connecting agents with tools, resources and prompt templates, and makes it easy to integrate AI agents with capabilities."
model = "gpt-4o-mini"

In [ ]:
async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

In [ ]:
async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, "My name's Ed. What do you know about me?")
    display(Markdown(result.final_output))

### Check the trace:

https://platform.openai.com/traces

### The 2nd type of MCP server - runs locally, calls a web service

### Brave Search - apologies - this will need another API key! But it's free again.

https://brave.com/search/api/

Set up your account, and put your key in the .env under `BRAVE_API_KEY`

In [6]:
env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}
params = {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": env}

async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()

In [ ]:
mcp_tools

In [8]:
instructions = "You are able to search the web for information and briefly summarize the takeaways."
request = f"Please research the latest news on Amazon stock price and briefly summarize its outlook. \
For context, the current date is {datetime.now().strftime('%Y-%m-%d')}"
model = "gpt-4o-mini"

In [ ]:
async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

### As usual, check out the trace:

https://platform.openai.com/traces

### And now the third time: running remotely

https://smithery.ai/server/@qubaomingg/stock-analysis-mcp

First we need to set up a free Share Price account with AlphaVantage at:

https://www.alphavantage.co/

Click "Get Free API Key"

And enter that in your .env as `ALPHA_VANTAGE_API_KEY`




In [2]:
import smithery

url = smithery.create_smithery_url("wss://server.smithery.ai/@qubaomingg/stock-analysis-mcp/ws", {
  "alphaVantageApiKey": os.getenv("ALPHA_VANTAGE_API_KEY")
})


#### Difficulties with the OpenAI Agents SDK SSE Client

This code should work (I believe) but it didn't!

```
params = {"url": url}
async with MCPServerSse(params=params) as server:
    mcp_tools = await server.list_tools()
```

So I wrote a quick client myself - see financial_datasets_client.py

In [3]:
from alpha_client import get_stock_tools_openai
openai_tools = await get_stock_tools_openai()

In [ ]:
openai_tools

In [13]:
instructions = "You can use tools to get stock prices."
request = f"Please let me know the share price of Amazon."
model = "gpt-4o-mini"

In [ ]:
agent = Agent(name="agent", instructions=instructions, model=model, tools=openai_tools)
with trace("conversation"):
    result = await Runner.run(agent, request)
display(Markdown(result.final_output))

### As usual, check out the trace:

https://platform.openai.com/traces

### Totally optional:

I was willing to fork out $20 to pay for this live financial data MCP server, at a cost of 1 cent per lookup!

But this is totally optional.

There's slightly more setup required, as we need to install their project locally to run their server:

https://mcp.so/server/mcp-server/financial-datasets?tab=content

You'll also need to set up an API key here:

https://www.financialdatasets.ai/

And then set up an env file in this repo with the name `.env` to contain your `FINANCIAL_DATASETS_API_KEY`

In [27]:

directory = "/Users/ed/projects/mcp-server"
env = {"FINANCIAL_DATASETS_API_KEY": os.getenv("FINANCIAL_DATASETS_API_KEY")}
params = {"command": "uv", "args": ["--directory", directory, "run", "server.py"], "env": env}


In [ ]:
async with MCPServerStdio(params=params) as server:
    mcp_tools = await server.list_tools()
mcp_tools

In [ ]:
async with MCPServerStdio(params=params) as mcp_server:
    agent = Agent(name="agent", instructions=instructions, model=model, mcp_servers=[mcp_server])
    with trace("conversation"):
        result = await Runner.run(agent, request)
    display(Markdown(result.final_output))

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/exercise.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Exercises</h2>
            <span style="color:#ff7800;">Explore MCP server marketplaces and integrate your own, using all 3 approaches.
            </span>
        </td>
    </tr>
</table>